In [ ]:
import re
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [210]:
player_link = []
team_link = []

Michael Jordan Trophy

In [211]:
for i in range(2020,2026):
    url = f"https://www.basketball-reference.com/awards/awards_{i}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find(id= "div_mvp")
    data = result.select("tbody > tr")
    MVP_data = []
    season = f"{i-1} - {i}"
    for j in data:
        rank = j.find("th", attrs={"data-stat": "rank"}).text.strip()
        name = j.find("td", attrs={"data-stat": "player"})
        player_link.append(name.find("a").get('href'))
        team_id = j.find("td", attrs={"data-stat": "team_id"})
        if team_id.find("a"):
            team_link.append(team_id.find("a").get('href'))
        data_dict = {
                    "Rank" : rank,
                    "Name" : name.text.strip(),
                    "Team_ID" : team_id.text.strip(),
                    "Season" : season
        }
        MVP_data.append(data_dict)
    data = pd.DataFrame(MVP_data)
    data.to_excel(f'MVP {season}.xlsx', index=False)

Champion Team

In [212]:
league_champ = []
for i in range(2020,2026):
    url = f"https://www.basketball-reference.com/leagues/NBA_{i}_totals.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')

    result = soup.find("div", id= "meta")
    p_list = result.find_all("p")
    league_champ.append(p_list[2].find("a").get('href'))


In [213]:
base_url = "https://www.basketball-reference.com"
for year in league_champ:
    response = requests.get(base_url+year)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find(id= "div_roster")
    players = result.select("tbody > tr")
    champ_data = []
    y = re.search(r"\d{4}", year).group(0)
    season = f"{int(y)-1}-{y}"
    team_id = re.search(r"\b\w{3}\b", year).group(0)
    for p in players:
        shirt_no = p.find("th", attrs={"data-stat": "number"}).text.strip()
        name = p.find("td", attrs={"data-stat": "player"})
        if name.find("a"):
            player_link.append(name.find("a").get('href'))
        experience = p.find("td", attrs={"data-stat": "years_experience"}).text.strip()
        data_dict = {"Shirt_No" : shirt_no,
                "Name" : name.text.strip(),
                "Experience" : experience,
                "Season" : season,
                 "Team_ID" : team_id }
        champ_data.append(data_dict)
    data = pd.DataFrame(champ_data)
    data.to_excel(f'League Champion {season}.xlsx', index=False)


Top 50 for each season

In [214]:
for i in range(2020,2026):
    url = f"https://www.basketball-reference.com/leagues/NBA_{i}_totals.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find(id= "div_totals_stats")
    top50 = result.select("tbody > tr")
    top50_data = []
    season = f"{i-1} - {i}"
    for j in top50:
        rank = j.find("th", attrs={"data-stat": "ranker"}).text.strip()
        name = j.find("td", attrs={"data-stat": "name_display"})
        if name.find("a"):
            player_link.append(name.find("a").get('href'))
        team_id = j.find("td", attrs={"data-stat": "team_name_abbr"})
        if team_id.find("a"):
            team_link.append(team_id.find("a").get('href'))
        data_dict = {
                    "Rank" : rank,
                    "Name" : name.text.strip(),
                    "Team_ID" : team_id.text.strip(),
                    "Season" : season
        }
        top50_data.append(data_dict)
        if int(rank) == 50 :
            break
    data = pd.DataFrame(top50_data)
    data.to_excel(f'Top 50 of {season}.xlsx', index=False)

Teams

In [ ]:
modify_link = []
for link in team_link :
    check = re.search(r"\/teams\/[A-Z]{3}\/", link).group(0)
    modify_link.append(check)

modify_link = list(set(modify_link))

In [ ]:
team_url = "https://www.basketball-reference.com"
teams_data = []
for link in modify_link:
    url = team_url+ link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find(id="meta")
    team_id = re.search(r"\b\w{3}\b", link).group(0)
    if result:
        name = result.find("h1").text.strip()
        location = result.find(string="Location:").next_element.text.strip()
        founded = result.find(string="Seasons:").next_element.text.strip()
        win_loss = result.find(string="Record:").next_element.text.strip()
        playoff = result.find(string="Playoff Appearances:").next_element.text.strip()
        championship = result.find(string="Championships:").next_element.text.strip()
    else:
        name = None
        location = None
        founded = None
        win_loss = None
        playoff = None
        championship = None
    

    data_dict = { "Team_ID" : team_id,
                 "Team Name" : name,
                 "Location": location,
                 "Founded Info" : founded,
                 "Win_Loss" : win_loss,
                 "playoff" : playoff,
                 "Championship": championship}
    teams_data.append(data_dict)
data = pd.DataFrame(teams_data)
data.to_excel(f'Team Information.xlsx', index=False)

Players

player_link = list(set(player_link))

In [ ]:
base_url = "https://www.basketball-reference.com"
player_data = []
for link in player_link:
    url = base_url+link
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    result = soup.find(id="meta")
    
    if result:
        name = result.find("h1").text.strip()
        strong_tag = result.find_all('strong')
        for tag in strong_tag :
            if "Position:" in tag.text:
                position = tag.next_sibling.strip()
            if "Shoots:" in tag.text:
                shoots = tag.next_sibling.strip()
            if "Experience:" in tag.text:
                experience = tag.next_sibling.strip()
        span_tag = result.select("p >span")
        height = span_tag[0].text.strip()
        weight = span_tag[1].text.strip()
        date_of_birth = span_tag[2].text.strip()
        town = span_tag[3].text.strip()
    else:
        name = None
        position = None
        shoots = None
        experience = None
        height = None
        weight = None
        date_of_birth = None
        town = None
    
    data_dict = {"Player Name" : name,
                 "Position" : position,
                 "Shoots": shoots,
                 "Experience" : experience,
                 "Height" : height,
                 "Weight" : weight,
                 "Date Of Birth": date_of_birth,
                 "Hometown" : town}
    player_data.append(data_dict)
data = pd.DataFrame(player_data)
data.to_excel('Players Information.xlsx', index=False)
